<a href="https://colab.research.google.com/github/ChowMeinFan/Classifying-Cells-Using-Machine-Learning/blob/main/ClassifyingCells(NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount our Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import necessary packages
import numpy as np
from tensorflow import keras
from keras import layers
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Nadam, Adamax, Lion, Adafactor
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from mlxtend.evaluate import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf

image_size = (1097, 1097)
classes = ['COS7', 'HEK293T', 'L', 'NC']

#Separate individual paths
train_path = '/content/drive/MyDrive/CSC308CellImagesV2/TrainingData'
test_path = '/content/drive/MyDrive/CSC308CellImagesV2/TestingData'
valid_path = '/content/drive/MyDrive/CSC308CellImagesV2/ValidationData'

In [ ]:
#Load the dataset of cell images
train_datagen = ImageDataGenerator(validation_split=0.25)

train_dataset = train_datagen.flow_from_directory(train_path, image_size, classes=classes, batch_size = 6)
valid_dataset = train_datagen.flow_from_directory(valid_path, image_size, classes=classes, batch_size = 3)
test_dataset = ImageDataGenerator().flow_from_directory(test_path, image_size, classes=classes, batch_size = 3)

Found 84 images belonging to 4 classes.
Found 12 images belonging to 4 classes.
Found 18 images belonging to 4 classes.


In [ ]:
#train_dataset = tf.reshape(train_dataset, (-1, 10))

In [ ]:
#def normalize_img(image, label):
#  """Normalizes images: `uint8` -> `float32`."""
#  return tf.cast(image, tf.float32) / 255., label

#train_dataset = train_dataset.map(
#    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
#train_dataset = train_dataset.cache()
##train_dataset = train_dataset.shuffle(ds_info.splits['train'].num_examples)
#train_dataset = train_dataset.batch(12)
#train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
#valid_dataset = valid_dataset.map(
#    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
#valid_dataset = valid_dataset.cache()
##train_dataset = train_dataset.shuffle(ds_info.splits['train'].num_examples)
#valid_dataset = valid_dataset.batch(12)
#valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
#test_dataset = test_dataset.map(
#    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
#test_dataset = test_dataset.cache()
##train_dataset = train_dataset.shuffle(ds_info.splits['train'].num_examples)
#test_dataset = test_dataset.batch(12)
#test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
#Function to track time, Initialize a callback object
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

In [ ]:
model = Sequential([
  keras.layers.InputLayer(input_shape=(1097, 1097, 3)),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(64, activation='relu'),
  keras.layers.Dense(4)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 3610227)           0         
                                                                 
 dense_8 (Dense)             (None, 128)               462109184 
                                                                 
 dense_9 (Dense)             (None, 128)               16512     
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 4)                 260       
                                                                 
Total params: 462134212 (1.72 GB)
Trainable params: 462134212 (1.72 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#Compile and Fit our Neural Network
model.compile( optimizer=Adam(learning_rate =.001),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

history = model.fit(
     x = train_dataset,
     validation_data = valid_dataset,
     epochs=100,
     callbacks=[cb],
     verbose=1)
print("Training Complete!")

Epoch 1/100


ResourceExhaustedError: ignored

In [ ]:
#Output our callback object
print(cb.logs)
print("Total Model Training Time:", sum(cb.logs))

In [ ]:
#Learning Curve
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 20])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
#Evaluate our model
score = model.evaluate(test_dataset, steps = 1, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 3.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

test_batches = ImageDataGenerator() \
                .flow_from_directory(test_path,
                                     target_size=(1097,1097),
                                     shuffle=False,
                                     batch_size=24)
test_imgs, test_labels = next(test_batches)
predictions = model.predict(test_batches, steps=1, verbose=0)
cm = confusion_matrix(test_labels.argmax(axis=1), predictions.argmax(axis=1), binary=False)
cm_plot_labels = classes
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')